In [1]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import os
from tensorflow.keras.preprocessing import image

Mounted at /content/drive


In [3]:
model = tf.keras.models.load_model(
    '/content/drive/MyDrive/Pneumonia_Project/Models/saved_models/densenet121_best_model.keras'
)
print("✅ Model loaded!")

# Fixed version - works with Keras 3.x
for layer in reversed(model.layers):
    try:
        shape = layer.output.shape
        if len(shape) == 4:
            last_conv_layer = layer.name
            print(f"✅ Last conv layer: {last_conv_layer}")
            break
    except:
        continue

✅ Model loaded!
✅ Last conv layer: densenet121


In [8]:
# Get the densenet121 base model
base_model = model.get_layer('densenet121')

# Find last conv layer inside it
for layer in reversed(base_model.layers):
    try:
        shape = layer.output.shape
        if len(shape) == 4:
            last_conv_layer = layer.name
            print(f"✅ Found conv layer: {last_conv_layer}")
            break
    except:
        continue

print(f"\n✅ Will use layer: {last_conv_layer}")

✅ Found conv layer: relu

✅ Will use layer: relu


In [13]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    base_model = model.get_layer('densenet121')

    grad_model = tf.keras.models.Model(
        inputs=base_model.inputs,
        outputs=[
            base_model.get_layer(last_conv_layer_name).output,
            base_model.output
        ]
    )

    with tf.GradientTape() as tape:
        inputs = tf.cast(img_array, tf.float32)
        conv_outputs, base_predictions = grad_model(inputs)
        class_channel = base_predictions[:, 0]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.nn.relu(heatmap)
    heatmap = heatmap.numpy()
    if heatmap.max() > 0:
        heatmap = heatmap / heatmap.max()

    # Fix - get final prediction from full model not base model
    final_pred = model(img_array, training=False)
    pred_score = float(final_pred.numpy()[0][0])

    return heatmap, pred_score


def save_gradcam_image(img_path, model, last_conv_layer_name, save_path, true_label):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded = np.expand_dims(img_array / 255.0, axis=0)

    heatmap, pred_score = make_gradcam_heatmap(
        img_array_expanded, model, last_conv_layer_name
    )

    pred_label = "PNEUMONIA" if pred_score > 0.260 else "NORMAL"
    confidence = pred_score if pred_score > 0.5 else 1 - pred_score

    original = np.uint8(img_array)
    heatmap_resized = cv2.resize(heatmap, (224, 224))
    heatmap_colored = cv2.applyColorMap(
        np.uint8(255 * heatmap_resized), cv2.COLORMAP_JET
    )
    heatmap_colored = cv2.cvtColor(heatmap_colored, cv2.COLOR_BGR2RGB)
    superimposed = cv2.addWeighted(original, 0.6, heatmap_colored, 0.4, 0)

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    correct = "✅ CORRECT" if pred_label == true_label else "❌ WRONG"
    fig.suptitle(
        f'Grad-CAM | True: {true_label} | Predicted: {pred_label} '
        f'({confidence*100:.1f}%) | {correct}',
        fontsize=13, fontweight='bold'
    )

    axes[0].imshow(original)
    axes[0].set_title('Original X-Ray', fontsize=12)
    axes[0].axis('off')

    axes[1].imshow(heatmap_resized, cmap='jet')
    axes[1].set_title('Heatmap (AI Focus Area)', fontsize=12)
    axes[1].axis('off')

    axes[2].imshow(superimposed)
    axes[2].set_title('Overlay', fontsize=12)
    axes[2].axis('off')

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()

    print(f"✅ Saved | True: {true_label} | Predicted: {pred_label} "
          f"| Confidence: {confidence*100:.1f}%")

In [10]:
# Print all layer names inside densenet121
base_model = model.get_layer('densenet121')
for layer in base_model.layers[-10:]:
    print(layer.name)

conv5_block15_concat
conv5_block16_0_bn
conv5_block16_0_relu
conv5_block16_1_conv
conv5_block16_1_bn
conv5_block16_1_relu
conv5_block16_2_conv
conv5_block16_concat
bn
relu


In [11]:
# Set the correct layer name
last_conv_layer = 'conv5_block16_concat'
print(f"✅ Using layer: {last_conv_layer}")

✅ Using layer: conv5_block16_concat


In [14]:
SAVE_DIR = '/content/drive/MyDrive/Pneumonia_Project/Models/results/gradcam'
os.makedirs(SAVE_DIR, exist_ok=True)

TEST_PATH = '/content/drive/MyDrive/Pneumonia_Project/Data/test'

pneumonia_imgs = os.listdir(f'{TEST_PATH}/PNEUMONIA')[:3]
normal_imgs    = os.listdir(f'{TEST_PATH}/NORMAL')[:3]

test_images = (
    [(f'{TEST_PATH}/PNEUMONIA/{f}', 'PNEUMONIA') for f in pneumonia_imgs] +
    [(f'{TEST_PATH}/NORMAL/{f}',    'NORMAL')    for f in normal_imgs]
)

print(f"🔍 Generating Grad-CAM for {len(test_images)} images...")
print()

for i, (img_path, true_label) in enumerate(test_images):
    save_path = f'{SAVE_DIR}/gradcam_{i+1}_{true_label}.png'
    save_gradcam_image(img_path, model, last_conv_layer, save_path, true_label)

print()
print("✅ All Grad-CAM images saved to Drive!")
print(f"📁 Location: {SAVE_DIR}")

Output hidden; open in https://colab.research.google.com to view.